In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Dropout, GlobalAveragePooling2D
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [26]:
def evaluate_model(model):
    train_loss, train_accuracy = model.evaluate(orig_train_data, verbose=2)
    print(f"Train Loss: {train_loss}")
    print(f"Train Accuracy: {train_accuracy}")

    val_loss, val_accuracy = model.evaluate(val_data, verbose=2)
    print(f"Validation Loss: {val_loss}")
    print(f"Validation Accuracy: {val_accuracy}")

    test_loss, test_accuracy = model.evaluate(test_data, verbose=2)
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

def plot_metrics(history, metric):
    plt.plot(history.history[metric], label='Train accuracy')
    plt.plot(history.history['val_'+metric], label='Validation accuracy')
    plt.legend()
    plt.show()

# Loading data from dataset using ImageDataGenerator
- Data augmentation is introduced in the pipeline for loading the train data. This helps to combat overfitting (as seen in the sample model in https://www.kaggle.com/code/ekanemgodwin/fruit-classification-test)

In [7]:
train_dir = "/kaggle/input/fruits-classification/Fruits Classification/train"
train_data_gen = ImageDataGenerator(rescale=1/255,
                                rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

train_data = train_data_gen.flow_from_directory(train_dir,
                           batch_size=64,
                           class_mode='categorical',
                           target_size=(150, 150))

Found 9700 images belonging to 5 classes.


In [18]:
#Train data without augmentation (for evaluation purpose down the line)
orig_train_data_gen = ImageDataGenerator(rescale=1/255)
orig_train_data = orig_train_data_gen.flow_from_directory(train_dir,
                           batch_size=64,
                           class_mode='categorical',
                           target_size=(150, 150))

Found 9700 images belonging to 5 classes.


In [8]:
val_dir = "/kaggle/input/fruits-classification/Fruits Classification/valid"
val_data_gen = ImageDataGenerator(rescale=1/255)

val_data = val_data_gen.flow_from_directory(val_dir,
                                           batch_size=32,
                                           class_mode='categorical',
                                           target_size=(150, 150))

Found 200 images belonging to 5 classes.


In [9]:
test_dir = "/kaggle/input/fruits-classification/Fruits Classification/test"
test_data_gen = ImageDataGenerator(rescale=1/255)
test_data = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=32,
                                             class_mode='categorical',
                                             target_size=(150, 150))

Found 100 images belonging to 5 classes.


# Model1:
- Experimenting with more layers compared to sample model (https://www.kaggle.com/code/ekanemgodwin/fruit-classification-test) to see performance

In [ ]:
model1 = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
])

model.summary()

In [ ]:
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history1 = model1.fit(train_data,
         epochs=30,
         validation_data=val_data,
         verbose=2)

In [ ]:
model1.save('model1.keras')

In [21]:
model1 = load_model("/kaggle/working/model1.keras")
evaluate_model(model1)

152/152 - 14s - 91ms/step - accuracy: 0.7886 - loss: 0.5552
Train Loss: 0.5552327036857605
Train Accuracy: 0.788556694984436
7/7 - 0s - 41ms/step - accuracy: 0.7400 - loss: 0.7044
Validation Loss: 0.704437255859375
Validation Accuracy: 0.7400000095367432
4/4 - 0s - 41ms/step - accuracy: 0.7700 - loss: 0.6008
Test Loss: 0.6007728576660156
Test Accuracy: 0.7699999809265137


From the results above, having trained for longer epochs with the larger model architecture, the train accuracy (78%) doesn't differ as much from validation accuracy (74%), showing effect of data augmentation in preventing overfitting

# Experimenting with MobileNetV2 (Transfer Learning)
- The feature extraction layers will be loaded and frozen (the top layers are excluded)
- Extra layers are added, tailored towards our application

In [4]:
base_model = MobileNetV2(input_shape = (150, 150, 3),
                  weights='imagenet',
                  include_top=False)
base_model.trainable = False

/tmp/ipykernel_34/1548143176.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape = (150, 150, 3),


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
inputs = Input(shape=(150, 150, 3))
x = base_model(inputs)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(5, activation='softmax')(x)

model2 = Model(inputs=inputs, outputs=outputs)
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ModelCheckpoint("model2.keras", monitor='val_accuracy', save_best_only=True)
]
history2 = model2.fit(train_data,
         epochs=50,
         validation_data=val_data,
         callbacks=callbacks,
         verbose=2)

In [22]:
model2 = load_model("/kaggle/working/model2.keras")
evaluate_model(model2)

W0000 00:00:1722631620.031734      80 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


152/152 - 19s - 128ms/step - accuracy: 0.9371 - loss: 0.2033
Train Loss: 0.2032615840435028
Train Accuracy: 0.9371134042739868
7/7 - 4s - 623ms/step - accuracy: 0.8650 - loss: 0.3868
Validation Loss: 0.3867763876914978
Validation Accuracy: 0.8650000095367432
4/4 - 4s - 975ms/step - accuracy: 0.9000 - loss: 0.3111
Test Loss: 0.311073362827301
Test Accuracy: 0.8999999761581421


W0000 00:00:1722631643.592491      80 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


From the above results, there is a significant increase in accuracy. 
- However, in the above model, a `Flatten` layer was used to reduce the dimensions from the feature extraction layers (MobileNetV2 layers). 
- The model below will use a `GlobalAveragePooling2D` layer to reduce the dimensions (an experiment to see performace, referenced from https://www.kaggle.com/code/utkarshsaxenadn/fruit-classification-mobilenetv2-acc-95)
- A dropout layer is added before final layer to also combat overfitting
- Callbacks are implemented, monitoring validation accuracy for 5 epochs and stopping training if there is no improvement in that time

In [ ]:
inputs = Input(shape=(150, 150, 3))
x = base_model(inputs)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(5, activation='softmax')(x)

model3 = Model(inputs=inputs, outputs=outputs)
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.summary()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ModelCheckpoint("model3.keras", monitor='val_accuracy', save_best_only=True)
]
history3 = model3.fit(train_data,
         epochs=50,
         validation_data=val_data,
         callbacks=callbacks,
         verbose=2)

In [28]:
model3 = load_model("/kaggle/working/model3.keras")
evaluate_model(model3)

W0000 00:00:1722632313.502337      80 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


152/152 - 18s - 120ms/step - accuracy: 0.9085 - loss: 0.2523
Train Loss: 0.2523084878921509
Train Accuracy: 0.9084535837173462


W0000 00:00:1722632329.665823      78 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


7/7 - 3s - 496ms/step - accuracy: 0.9100 - loss: 0.2768
Validation Loss: 0.27681341767311096
Validation Accuracy: 0.9100000262260437
4/4 - 2s - 424ms/step - accuracy: 0.8800 - loss: 0.2847
Test Loss: 0.2847316861152649
Test Accuracy: 0.8799999952316284


The use of `GlobalAveragePooling2D` layer seems to have a slightly better performance 